In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.cross_decomposition import PLSRegression

from Strategy import *

In [2]:
temp = []

for i in range(4):
    gc_res = pd.read_csv("Granger_Causality_Res%s.csv" % i)
    temp.append(gc_res)

In [3]:
pred_res = pd.concat(temp, axis=0)

In [8]:
pred_res.sort_values(['ACC_PLS'], ascending=False)

,Unnamed: 0,MSE_PLS,ACC_PLS,MSE_AVG,ACC_AVG
34,PAR_Open,4.716669,0.729167,2.765642,0.520833
75,BRC_Open,35.017132,0.729167,1238.044656,0.479167
57,FMBI_Open,93.213371,0.708333,198.053992,0.500000
30,CEY_Open,46.210416,0.687500,3051.817179,0.541667
84,XEL_Open,34.931812,0.687500,15.654795,0.604167
...,...,...,...,...,...
29,VRTV_Open,55.040428,0.375000,2.538731,0.354167
42,HNI_Open,126.506448,0.375000,37381.889251,0.395833
10,PSB_Open,97.518874,0.354167,37570.645351,0.500000
11,JBSS_Open,54.202419,0.333333,37581.266664,0.458333


In [9]:
df = pd.read_csv("Y:\\Dropbox\\Dropbox (MIT)\\Robinhood Trading\\Stock Data\\broader_stock.csv")
df = df.set_index(pd.to_datetime(df['Date']))
df.drop(['Date'], axis=1, inplace=True)
pct_df = df.pct_change().shift(1).iloc[2:]

In [20]:
leader_tick_dict2 = {}
w_mktre = (1 + pct_df['SPY_Close']).resample('W').prod() - 1
tick = "PAR_Open"
target_arr = pct_df[tick].dropna()
w_target = (1 + target_arr).resample('W').prod() - 1
Y = w_target.shift(-1)
leader_set = []

for leader in pct_df.columns[::3]:
    if leader != tick:
        leader_arr = pct_df[leader].dropna()
        w_leader = (1 + leader_arr).resample('W').prod() - 1

        tempreg_dta = pd.concat([Y, w_target, w_mktre, w_leader], axis=1).dropna()
        tempreg_dta.columns = ['Y', 'Y-1', 'Mkt', 'Lead']

        if tempreg_dta.shape[0] >= 36 * 4:
            ols = sm.OLS(tempreg_dta['Y'].iloc[-36 * 4:],
                         sm.add_constant(tempreg_dta[['Y-1', 'Mkt', 'Lead']].iloc[-36 * 4:]))
            res = ols.fit(cov_type='HC0')
            leader_sig = res.pvalues[3]

        elif tempreg_dta.shape[0] >= 12 * 4:
            ols = sm.OLS(tempreg_dta['Y'].iloc[-12 * 4:],
                         sm.add_constant(tempreg_dta[['Y-1', 'Mkt', 'Lead']].iloc[-12 * 4:]))
            res = ols.fit(cov_type='HC0')
            leader_sig = res.pvalues[3]

        else:
            leader_sig = 1

        if leader_sig <= 1e-3:
            leader_set.append(leader)

leader_tick_dict2[tick] = leader_set

In [16]:
leader_tick_dict

{'PAR_Open': ['MDC_Open',
  'UNFI_Open',
  'LYV_Open',
  'HRL_Open',
  'CBSH_Open',
  'AIR_Open',
  'SWN_Open',
  'GIS_Open',
  'KODK_Open',
  'WDC_Open',
  'EXPR_Open',
  'STAR_Open',
  'PMT_Open',
  'SCL_Open',
  'CAR_Open',
  'SJM_Open',
  'OZK_Open',
  'KHC_Open',
  'K_Open']}

In [21]:
leader_tick_dict2

{'PAR_Open': ['WWE_Open',
  'ASIX_Open',
  'WH_Open',
  'LW_Open',
  'VVV_Open',
  'JBGS_Open',
  'FCPT_Open',
  'ALT_Open',
  'HRB_Open']}